In [6]:
import argparse

import torch
import uuid

import numpy as np

from torch_geometric.nn import Node2Vec
from torch_geometric.utils import erdos_renyi_graph

import os
import csv

import gph
import gtda
from gtda import diagrams
from gtda import homology

import matplotlib.pyplot as plt

import sys
# appending a path
sys.path.append('../rawos')
import metrics
from sklearn.metrics import pairwise_distances
import scipy
from scipy import special
from scipy.stats import pearsonr
import skbio

In [20]:
filename1 ='../results/er/er-c5-d16-l5-n10-p0_25-keep-0a05984ca01c41d495dd131e2eb636eb.tsv'
filename2 = '../results/er/er-c5-d16-l5-n10-p0_25-keep-0cb8d19dd9df4df7b80c46432c477b07.tsv'
X= np.loadtxt(filename1, delimiter="\t")
diamx = metrics.diameter(X)
X= X/diamx

Y= np.loadtxt(filename2, delimiter="\t")
diamy = metrics.diameter(Y)
Y= Y/diamy

dX = pairwise_distances(X, metric='euclidean')
dY = pairwise_distances(Y, metric='euclidean')

DX = (dX+dX.T)/2
DY = (dY+dY.T)/2
 
DDX= skbio.stats.distance.DistanceMatrix(DX)
DDY= skbio.stats.distance.DistanceMatrix(DY)

mantel= skbio.stats.distance.mantel(DDX,DDY, method='spearman', permutations=10, alternative='two-sided', strict=True, lookup=None)



In [40]:
mantel[1]

0.36363636363636365

In [47]:
def Mantel_test(filename1, filename2, method = 'pearson', permutations = 100): 
    """Computes the correlation of two distance matrices of same size via the Mantel test
        permutations = number of permutations performed for the test 
        method = 'pearson' or 'spearman' corrrelation coefficient
    """
    X= np.loadtxt(filename1, delimiter="\t")
    diamx = metrics.diameter(X)
    X= X/diamx

    Y= np.loadtxt(filename2, delimiter="\t")
    diamy = metrics.diameter(Y)
    Y= Y/diamy

    dX = pairwise_distances(X, metric='euclidean')
    dY = pairwise_distances(Y, metric='euclidean')
    
    ## make sure they are symmetric because package seems sensitive to slight assymetries.. 
    dX = (dX+dX.T)/2
    dY = (dY+dY.T)/2
    ## fit the package fomrmat of Distance 
    DDX= skbio.stats.distance.DissimilarityMatrix(dX)
    DDY= skbio.stats.distance.DissimilarityMatrix(dY)
    
    Mantel = skbio.stats.distance.mantel(DDX,DDY, method=method, permutations=permutations, alternative='two-sided', strict=True, lookup=None)
    corr = Mantel[0]
    pval = Mantel[1]
    
    return corr, pval

In [49]:
Mantel_test(filename1,filename2, 'spearman',1000)

(0.07610620217403695, 0.1918081918081918)